In [1]:
%matplotlib  inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from plot_animate import frequency, plot_mol_mode

In [2]:
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

three body structure (one dimension)
<img src="./figure/co2.png" alt="Drawing" style="width: 200px;"/>
https://en.wikipedia.org/wiki/Carbon_dioxide

In [3]:
# System CO2#

'''
O1:1
C:2
O2:3
'''

# Stiffness #
KCO1 = 1600
KCO2 = 1600

stiffness_list = np.array([KCO1,KCO2])
#spring_length_list = np.array([LOH1,L0H2])


# Mass (index: 1,2....n) #
MO1 = 16
MC = 12
MO2 = 16


mass_list = np.array([MO1,MC,MO2])

# Position (equilibrium length)#
PO1 = [0,0]
PC = [1.163,0]
PO2 = [2.326,0]


position_list = np.array([PO1,PC,PO2])

# Bond list #

bond_list = np.array([[0,1],[1,2]])


In [4]:
def bond_stiffness_1D(k,m1,m2):
    return np.array([[k/(m1**0.5*m1**0.5),-k/(m1**0.5*m2**0.5)],[-k/(m1**0.5*m2**0.5),k/(m2**0.5*m2**0.5)]])

In [5]:
def hessian_1D(m_list,p_list,b_list,k_list):
    hm = np.zeros((m_list.shape[0],m_list.shape[0]))
    
    num_cout = 0
    for i,j in bond_list:
        k_n = bond_stiffness_1D(k_list[num_cout],m_list[i],m_list[j])
        
        hm[i,i] += k_n[0,0]
        hm[i,j] += k_n[0,1]
        
        hm[j,i] += k_n[1,0]
        hm[j,j] += k_n[1,1]
        
    return hm

In [6]:
system_kmatrix_co2 = hessian_1D(mass_list,position_list,bond_list,stiffness_list)

In [7]:
# hessian matrix (stiffness matrix)
print(system_kmatrix_co2)

[[ 100.         -115.47005384    0.        ]
 [-115.47005384  266.66666667 -115.47005384]
 [   0.         -115.47005384  100.        ]]


In [8]:
# solve eignvalue and eignvector #
system_co2_val, system_co2_vec = np.linalg.eig(system_kmatrix_co2)
print(system_co2_val)
print(system_co2_vec)

[ 3.66666667e+02  1.00000000e+02 -2.45876543e-15]
[[-3.69274473e-01 -7.07106781e-01  6.03022689e-01]
 [ 8.52802865e-01  2.60513789e-16  5.22232968e-01]
 [-3.69274473e-01  7.07106781e-01  6.03022689e-01]]


Frequency
<img src="./figure/freq.png" alt="Drawing" style="width: 200px;"/>
https://www.colby.edu/chemistry/PChem/notes/NormalModesText.pdf

In [9]:
modeshape = 1
plot_mol_mode(position_list,frequency(system_co2_val[modeshape]),system_co2_vec[:,modeshape])

three body structure (two dimensions)
<img src="./figure/question.png" alt="Drawing" style="width: 200px;"/>
https://www.khanacademy.org/science/ap-biology/chemistry-of-life/structure-of-water-and-hydrogen-bonding/a/hs-water-and-life-review

In [10]:
# System H2O# 
# strength and bending #

'''
H1:1
H2:2
O:3
'''

# Stiffness #
KOH1 = 10
KOH2 = 10

#LOH1 = 0.5
#LOH2 = 0.5

stiffness_list = np.array([KOH1,KOH2])
#spring_length_list = np.array([LOH1,L0H2])


# Mass #
MH1 = 1
MH2 = 1
MO = 16

mass_list = np.array([MH1,MH2,MO])

# Position #
PH1 = [0,0]
PH2 = [2,0]
PO = [0.5,1.5]

position_list = np.array([PH1,PH2,PO])

# Bond list #

bond_list = np.array([[0,2],[1,2]])


In [11]:
def bond_stiffness_2D(k,m1,m2):
    return np.array([[k/(m1**0.5*m1**0.5),0,-k/(m1**0.5*m2**0.5),0],[0,k/(m1**0.5*m1**0.5),0,-k/(m1**0.5*m2**0.5)],[-k/(m1**0.5*m2**0.5),0,k/(m2**0.5*m2**0.5),0],[0,-k/(m1**0.5*m2**0.5),0,k/(m2**0.5*m2**0.5)]])

In [12]:
def hessian_2D(m_list,p_list,b_list,k_list):
    hm = np.zeros((m_list.shape[0]*2,m_list.shape[0]*2))
    
    num_cout = 0
    for i,j in bond_list*2:

        k_n = bond_stiffness_2D(k_list[num_cout],m_list[int(i/2)],m_list[int(j/2)])
        #print(k_n)
        hm[i,i:i+2] += k_n[0,0:2]
        hm[i,j:j+2] += k_n[0,2:4]
        
        hm[i+1,i:i+2] += k_n[1,0:2]
        hm[i+1,j:j+2] += k_n[1,2:4]
        
        hm[j,i:i+2] += k_n[2,0:2]
        hm[j,j:j+2] += k_n[2,2:4]
        
        hm[j+1,i:i+2] += k_n[3,0:2]
        hm[j+1,j:j+2] += k_n[3,2:4]
        
 
    return hm

In [13]:
system_kmatrix = hessian_2D(mass_list,position_list,bond_list,stiffness_list)

In [14]:
print(system_kmatrix)

[[10.    0.    0.    0.   -2.5   0.  ]
 [ 0.   10.    0.    0.    0.   -2.5 ]
 [ 0.    0.   10.    0.   -2.5   0.  ]
 [ 0.    0.    0.   10.    0.   -2.5 ]
 [-2.5   0.   -2.5   0.    1.25  0.  ]
 [ 0.   -2.5   0.   -2.5   0.    1.25]]


In [15]:
system_val, system_vec = np.linalg.eig(system_kmatrix)

In [16]:
system_vec[:,4]

array([ 0.00000000e+00,  7.07106781e-01,  0.00000000e+00, -7.07106781e-01,
        0.00000000e+00, -6.61592747e-16])